# Predicting sentiment from product reviews


This first notebook explores logistic regression and feature engineering with existing GraphLab functions.

Use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

## Imports

In [38]:
from __future__ import division
import graphlab
import math
import string
import numpy as np

# Data preperation

Data set consists of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create is assigned to damiansp@gmail.com and will expire on March 07, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-29 16:18:15,306 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1464553093.log


Preview data set.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

View a specific example of a baby product.

In [4]:
products[1976]

{'name': 'Playtex Drop-Ins Original BPA Free Nurser Newborn Starter Set',
 'rating': 4.0,
 'review': 'This kit has everything you need to start your baby off on disposable bottles. Assorted sizes and nipples that give you and your baby a great variety.'}

Data transformations:

1. Remove punctuation
2. Transform the reviews into word-counts.

**Aside**. In this notebook, all punctuation is removed for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. Additionally, more advanced tokenization may be done, so that, for example, derived endings (like the plural -s/-es) are removed so that, for example *diaper* and *diapers* are tabulated together, and case should probably be ignored for most words, etc.

In [5]:
def remove_punctuation(text):
    # requires string module
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products[1976]['word_count']

{'a': 1,
 'and': 2,
 'assorted': 1,
 'baby': 2,
 'bottles': 1,
 'disposable': 1,
 'everything': 1,
 'give': 1,
 'great': 1,
 'has': 1,
 'kit': 1,
 'need': 1,
 'nipples': 1,
 'off': 1,
 'on': 1,
 'sizes': 1,
 'start': 1,
 'that': 1,
 'this': 1,
 'to': 1,
 'variety': 1,
 'you': 2,
 'your': 2}

## Extract sentiments

**Ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : 1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 3, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3, 'ingenious':1, 'love': 2, 'what': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'all': 2,'help': 1, 'cried': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'had': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'fantastic':1, 'help': 1, 'give': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1, 'standarad':1, 'another': 1, 'when': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2, 'nannys': 1,'just': 1, 'food': 1, ...",1


Data set now contains an extra column called **sentiment** which is either positive (1) or negative (-1).

## Split data into training and test sets

In [9]:
train_data, test_data = products.random_split(0.8, seed = 1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

Use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. 

In [10]:
sentiment_model = graphlab.logistic_classifier.create(
    train_data, target = 'sentiment', features = ['word_count'], validation_set = None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 4.736617     | 0.960964          |

| 6         | 13       | 3.000000  | 5.131912     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [11]:
sentiment_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 121713
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 121712

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : lbfgs
Solver iterations             : 6
Solver status                 : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)           : 5.5359

Settings
--------
Log-likelihood                : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]      : 41.9847
word_count[placeid]           : 41.7354
word_count[labelbox]          : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid]: 40.0454
word_count[knobskeeping]      : 36.2091

Lowest Negative Coefficients
-----------

**Aside**: the warning "Terminated due to numerical difficulties --- this model may not be ideal" means that the quality metric failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, (covered later). Regularization lessens the effect of extremely rare words. For now, the issue is ignored.

Extract the weights (coefficients) as an SFrame:

In [13]:
weights = sentiment_model.coefficients
print weights.column_names()
print len(weights)

['name', 'index', 'class', 'value', 'stderr']
121713


There are a total of `121713` coefficients in the model. Positive weights ($w_j$) correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Calculate how many *weights* are negative and how many are non-negative

In [14]:
weights[:10]

name,index,class,value,stderr
(intercept),None,1,1.30337080544,None
word_count,recommend,1,0.303815600015,None
word_count,moist,1,0.671556821414,None
word_count,osocozy,1,0.426326525702,None
word_count,keps,1,7.3963370872,None
word_count,leak,1,-0.24658014554,None
word_count,holder,1,-0.0300523581012,None
word_count,was,1,-0.0530004786379,None
word_count,now,1,0.0383787882079,None
word_count,wipe,1,0.165506649337,None


In [14]:
num_positive_weights = sum(weights['value'] >= 0)
num_negative_weights = sum(weights['value'] < 0)

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


## Making predictions with logistic regression

Now that a model is trained, make predictions on the **test data**. In this section, explore 3 examples in the test data set: **sample_test_data**.

In [15]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2, 'all': 1,'love': 1, 'purchased': ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1, 'would': 2,'almost': 1, 'decals' ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1, 'money': 1,'into': 1, 'back': 1, ...",-1


Examine first row of the **sample_test_data**. Here's the full review:

In [16]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is negative.

In [17]:
sample_test_data[2]['review']

"Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it."

Make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **1** if the sentiment is positive and **-1** if the sentiment is negative. The **score** (or **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  Obtain the **scores** using GraphLab Create. For each row, the **score** is a number on **[-inf, inf]**.

In [18]:
scores = sentiment_model.predict(sample_test_data, output_type = 'margin')
print scores

[6.734619727059167, -5.734130996760246, -14.668460404468407]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Calculate $\hat{y}$, the class predictions:

In [19]:
scores2 = scores.apply(lambda scores : 1 if scores >= 0 else -1)

Verify that the class predictions obtained by calculations are the same as that obtained from GraphLab Create.

In [20]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)
print scores2

Class predictions according to GraphLab Create:
[1, -1, -1]
[1, -1, -1]


### Probability predictions

Calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, calculate the probability that a sentiment is positive using the above formula.

In [21]:
p1 = scores.apply(lambda scores: 1. / (1. + math.exp(-scores)))
p1

dtype: float
Rows: 3
[0.9988123848377196, 0.0032232681818007173, 4.261557996655897e-07]

Test: compare probability predictions with ones obtained from GraphLab Create.

In [22]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377196, 0.003223268181800717, 4.261557996655895e-07]


# Find the most positive and negative reviews

Examining the full test dataset, **test_data**, use GraphLab Create to form predictions on all of the test data points.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. These are called "most positive reviews."

In [23]:
pred_test = sentiment_model.predict(test_data, output_type = 'probability')
#pred_test
test_data['predictions'] = pred_test
test_data.head()

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1, 'standarad':1, 'another': 1, 'when': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2, 'nannys': 1,'just': 1, 'food': 1, ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,"{'and': 1, 'babys': 1,'love': 1, 'like': 1, ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'and': 3, 'all': 1,'later': 1, 'reference': ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'and': 2, 'because': 1,'just': 1, 'less': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'infant': 1, 'being': 1,'all': 1, 'course': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'and': 1, 'own': 1,'it': 3, 'our': 1, 'f ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'and': 3, 'rating': 1,'have': 1, 'just': 1, ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'and': 2, 'beautiful':1, 'love': 1, 'elmo': 1, ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'remembering': 1, 'and':4, 'year': 1, 'am': 1, ...",1


In [24]:
best20 = test_data.topk('predictions', k = 20)
best20.print_rows(num_rows = 20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Munchkin Mozart Magic Cube  | My wife and I have been li... |  4.0   |
|  BABYBJORN Potty Chair - Red  | Our family is just startin... |  5.0   |
| Safety 1st Tot-Lok Four Lo... | I have a wooden desk that ... |  5.0   |
| Summer Infant Complete Nur... | This Nursery and Bath Care... |  4.0   |
| Leachco Snoogle Total Body... | I have had my Snoogle for ... |  5.0   |
| HALO SleepSack Micro-Fleec... | I love the Sleepsack weara... |  5.0   |
| Peg Perego Primo Viaggio C... | We have been using this se... |  5.0   |
|   Capri Stroller - Red Tech   | First let me say that I wa... |  4.0   |
| Wizard Convertible Car Sea... | My son was born big and re... |  5.0   |
| Britax Marathon Convertibl... | My son began using the Mar... |  5.0   |
| Britax Decathlon Conver

In [25]:
worst20 = test_data.topk('predictions', k = 20, reverse = True)
worst20.print_rows(num_rows = 20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

## Compute accuracy of the classifier

Evaluate the accuracy of the trained classifer, given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

In [26]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    n_correct = sum([p == t for (p, t) in zip(predictions, true_labels)])

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = n_correct / len(predictions)
    
    return accuracy

Compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [27]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

## Learn another classifier using fewer words

There were *a lot* of words in the model we trained above. Train a simpler logistic regression model using only a subet of words that occur in the reviews, namely the following 20:

In [28]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 
                     'less', 'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return']

In [29]:
len(significant_words)

20

Use the **word_count** column and trim out all words that are **not** in the **significant_words** list above on both the training and test set.

In [30]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude = False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude = False)

Example review:

In [31]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column from before looks like:

In [32]:
print train_data[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Using only a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [33]:
print train_data[0]['word_count_subset']

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

Build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [34]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set = None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.552971     | 0.866815          |

| 6         | 7        | 0.656503     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 21
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 20

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 6
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.6771

Settings
--------
Log-likelihood                : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]      : 1.6773
word_count_subset[perfect]    : 1.5145
word_count_subset[love]       : 1.3654
(intercept)                   : 1.2995
word_count_subset[easy]       : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed]: -2.3551
word_count_subset[ret

Compute the classification accuracy using the `get_classification_accuracy` function above.

In [35]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Inspect the weights (coefficients) of the **simple_model**:

In [36]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,great,1,0.94469126948,0.0209509926591
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,work,1,-0.621700012425,0.0230330597946


Sort the coefficients by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [37]:
simple_mod_coef_table = simple_model.coefficients.sort('value', ascending = False).print_rows(num_rows = 21)
simple_mod_coef_table

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

In [39]:
sentiment_mod_coef_table = sentiment_model.coefficients.sort('value', ascending = False)
words = sentiment_mod_coef_table['index']
sig_inds = [w in significant_words for w in words]
sig_inds = np.array(sig_inds)
sig_inds = np.where(sig_inds == True)
sig_inds = sig_inds[0]
for i in sig_inds:
    print 'i:', i
    print sentiment_mod_coef_table[i]

i: 24006
{'index': 'perfect', 'stderr': None, 'name': 'word_count', 'value': 1.751901143919695, 'class': 1}
i: 26933
{'index': 'loves', 'stderr': None, 'name': 'word_count', 'value': 1.566485175695283, 'class': 1}
i: 29128
{'index': 'love', 'stderr': None, 'name': 'word_count', 'value': 1.4330168543924964, 'class': 1}
i: 31144
{'index': 'great', 'stderr': None, 'name': 'word_count', 'value': 1.314592450385744, 'class': 1}
i: 33087
{'index': 'easy', 'stderr': None, 'name': 'word_count', 'value': 1.2134693782158015, 'class': 1}
i: 45594
{'index': 'little', 'stderr': None, 'name': 'word_count', 'value': 0.6741624574993091, 'class': 1}
i: 46951
{'index': 'well', 'stderr': None, 'name': 'word_count', 'value': 0.6279648775667174, 'class': 1}
i: 60681
{'index': 'car', 'stderr': None, 'name': 'word_count', 'value': 0.19526367061771457, 'class': 1}
i: 61421
{'index': 'able', 'stderr': None, 'name': 'word_count', 'value': 0.17433127255184858, 'class': 1}
i: 67982
{'index': 'old', 'stderr': None,

# Comparing models

Compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [40]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [41]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

Repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [42]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [43]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

## Baseline: Majority class prediction

It is common to use the **majority class classifier** as the a baseline model for comparison with classifier models. The majority classifier model predicts the majority class for all data points. Models should at least beat the majority class classifier.

Get the majority class in the **train_data**?

In [44]:
num_positive  = (train_data['sentiment'] == 1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Compute the accuracy of the majority class classifier on **test_data**.

In [45]:
num_positive / len(train_data['sentiment'])

0.8407087605684476